# Notebook Setup

<a target="_blank" href="https://colab.research.google.com/github/PacktPublishing/Generative-AI-Integration-Patterns-1E/blob/main/Chapter07/Integration_pattern_Real_time_intent_classification.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
#Install dependencies

!pip install --upgrade google-cloud-aiplatform

In [ ]:
#Authenticate
!gcloud auth application-default login

In [ ]:
import base64
import json

import vertexai
from vertexai.generative_models import GenerativeModel, Part, FinishReason
import vertexai.preview.generative_models as generative_models



In [ ]:
PROJECT = "testproject-410220"#@param {type:"string"}
LOCATION = "us-central1"#@param {type:"string"}
MODEL = "gemini-1.5-flash-001"#@param {type:"string"}

# Function Definitions

In [ ]:
generation_config = {
    "max_output_tokens": 8192,
    "temperature": 0,
    "top_p": 0.95,
}

safety_settings = {
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
}

def generate(prompt):
  vertexai.init(project=PROJECT, location=LOCATION)
  model = GenerativeModel(MODEL)
  responses = model.generate_content(
      [prompt],
      generation_config=generation_config,
      safety_settings=safety_settings,
      stream=False,
  )
  return(responses)


# Entry Point

In [ ]:
#In this case we will simulate the input from a chat interface

message = "I want to open an account"

# Prompt Preprocessing

In [ ]:
#In this section we define the prompt, as the task is to perform intent classification we will identify the intent by exposing the possible values to the LLM
prompt_template = """
You are a helpful assistant for an online financial services company that allows users to check their balances, invest in certificates of deposit (CDs), and perform other financial transactions.

Your task is to identify what your customers are trying to do and return a well formed JSON object.

1. Carefully analyze the content of the message.
2. Classify what the user is trying to do within these options:
    * New Account: The user is trying to sign up. Return {{"intent": "signup", "content":"null"}}
    * Change Password: The user needs to reset their password. Return {{"intent":"change_password", "content":"null"}}
    * Check Balance: The user needs to check their balance. Return {{"intent": "check_balance", "content":"null"}}
    * Invest in CD: The user wants to invest in a certificate of deposit. Return {{"intent": "invest_cd", "content": "Extract relevant information such as investment amount and term"}}
    * Withdraw Funds: The user wants to withdraw money. Return {{"intent": "withdraw_funds", "content": "Extract information like amount and withdrawal method"}}
    * Transfer Funds: The user wants to transfer money between accounts. Return {{"intent": "transfer_funds", "content": "Extract information like amount, source account, and destination account"}}
    * Account Information: The user wants to access or update their account information. Return {{"intent": "account_info", "content": "Identify the specific information the user needs"}}
    * Lost/Stolen Card: The user wants to report a lost or stolen card. Return {{"intent": "lost_card", "content": "null"}}
    * Support: The user needs help and is not sure what to do. Return {{"intent": "support", "content": "null"}}
    * Other: For other queries, politely decline to answer and clarify what you can help with.
3. Only return the proper JSON result from your classification.
4. Always think step by step.

User question: {query}
JSON:
"""


# Inference

In [ ]:
result = generate(prompt_template.format(query=message))

# Result Postprocessing

In [ ]:
### Sometimes model return markdown friendly content, in this case we will implement a function to filter this.

def extract_json(text):
  """
  Extracts the JSON portion from a string containing backticks.

  Args:
    text: The string containing JSON data within backticks.

  Returns:
    A dictionary representing the extracted JSON, or None if no valid JSON is found.
  """
  start_index = text.find("```json")
  end_index = text.find("```", start_index + 7)  # +7 to skip "```json"

  if start_index != -1 and end_index != -1:
    json_string = text[start_index + 7: end_index]  # Extract the JSON string
  else:
    json_string = text
  try:
    json_data = json.loads(json_string)
    return json_data
  except json.JSONDecodeError:
    return None

def process_intent(intent):
  if intent["intent"] == "signup":
    #If a user is trying to sign up you could redirect the to a sign up page for example.
    return("Sign up process")
  elif intent["intent"] == "change_password":
    #If a user is looking into changing their password, you could either do it through the chatbot, or redirect to a password change page.
    return("Change password")
  elif intent["intent"] == "check_balance":
    #In this case you could have a function that would query a database to obtain the balance (as long as the user is logged in or not)
    return("Check account balance")
  elif intent["intent"] == "invest_cd":
    #For the investment intent, this could redirect to a page where investment options can be selected.
    return("Invest in a CD")
  elif intent["intent"] == "withdraw_funds":
    return("Withdraw funds")
  elif intent["intent"] == "transfer_funds":
    return("Transfer funds")
  elif intent["intent"] == "account_info":
    return("Account information")
  elif intent["intent"] == "lost_card":
    return("Report lost card")
  elif intent["intent"] == "support":
    return("Contact support")
  elif intent["intent"] == "other":
    return("Other kind of intent")
  else:
    return("If a intent was classified as something else you should investigate what is going on.")

intent = process_intent(extract_json(result.text))

print(intent)

# Result Presentation

In [ ]:
#In this case we will use a Gradio interface to interact with the system

#Install Gradio

!pip install --upgrade gradio

In [ ]:
import gradio as gr

def chat(message, history):
    response = generate(prompt_template.format(query=message))
    intent_action = process_intent(extract_json(response.text))
    history.append((message, intent_action))
    return "", history


with gr.Blocks() as demo:
  gr.Markdown("Fintech Assistant")
  chatbot = gr.Chatbot(show_label=False)
  message = gr.Textbox(placeholder="Enter your question")
  message.submit(chat, [message, chatbot],[message, chatbot]  )

demo.launch(debug=True)